In [181]:
from sage.modules.free_module_integer import IntegerLattice


p = next_prime(2^16)
n = ceil(log(p, 2))
k = ceil(sqrt(n)) + ceil(log(n, 2))

In [182]:
def generate_pairs(N, q):
    pairs = []
    for _ in range(N):
        x_i = randint(1, q - 1)
        u_i = randint(0, q - 1)  # Considerando que u_i é um inteiro positivo
        pairs.append((x_i, u_i))
    return pairs

pairs = generate_pairs(n, p)
print(pairs)

[(12413, 7770), (4926, 1408), (20530, 662), (6217, 9634), (42950, 61554), (36743, 16258), (4254, 23285), (53120, 32693), (13825, 59565), (31212, 16822), (40282, 37965), (55723, 38358), (2606, 46247), (17496, 21129), (14753, 38846), (9554, 34652), (36765, 20682)]


In [183]:
def msb_k(y, p, B):
    return (y // B) % p



In [184]:
def build_integer_lattice(oracle_inputs):
    basis_vectors = []
    for i in range(n):
        p_vector = [0] * (n+1)
        p_vector[i] = p*p
        basis_vectors.append(p_vector)
    scaled_answers = list(map(lambda oi: oi*p, oracle_inputs))
    basis_vectors.append(scaled_answers + [1])
    return IntegerLattice(basis_vectors)

In [185]:
def lattice_reduction(G):
    return G.LLL()

In [186]:
def recover_hidden_number(G, u, p, l):
    alpha = 0
    d = G.ncols() - 1
    lambda_val = 2 ** k
    B = p // lambda_val
    for i in range(d):
        alpha += msb_k(u[i], l, B) * G[i, d]
    return alpha % p


In [187]:
l=40
t=[x for (x, _) in pairs]
u=[y for (_, y) in pairs]
# Step 1: Build the lattice
G = build_integer_lattice(t)

# Step 2: Apply lattice reduction algorithms
G_reduced = lattice_reduction(G)

# Step 3: Recover the hidden number
alpha = recover_hidden_number(G_reduced, u, p, n)

print("Recovered hidden number:", alpha)

Recovered hidden number: 36535


In [188]:
def verify_secret(s, x_values, u_values, p, k):
    lambda_val = 2 ** k
    B = p // lambda_val
    for x, u in zip(x_values, u_values):
        msb_k_calc = msb_k((s * x) , p, B)
        print("msb_k_calc", msb_k_calc)
        
        if msb_k_calc != u:
            return False
    return True

# Verificar o segredo extraído
is_correct = verify_secret(alpha, t, u, p, k)

if is_correct:
    print("O segredo extraído está correto.")
else:
    print("O segredo extraído está incorreto.")

msb_k_calc 8081
O segredo extraído está incorreto.
